In [1]:
import os
import sys
import numpy as np
import pandas as pd
from airflow.providers.mysql.hooks.mysql import MySqlHook
from airflow.providers.postgres.hooks.postgres import PostgresHook

warehouse_hook = PostgresHook(postgres_conn_id='airflow_warehouse_user', schema='ubuntu')
mifos_hook = MySqlHook(mysql_conn_id='mifos_db', schema='mifostenant-safaricom')
mifos1_hook = MySqlHook(mysql_conn_id='mifos_db', schema='mifostenant-default')
airflow_hook = MySqlHook(mysql_conn_id='airflow_server_admin', schema='monitoring')
middleware_hook = PostgresHook(postgres_conn_id='asante_mfs_middleware', schema='asante_datawarehouse')
jubilee_hook = MySqlHook(mysql_conn_id='mifos_db', schema='mifostenant-jubilee')

/tmp/ipykernel_28591/2123065259.py:8 DeprecationWarning: The "schema" arg has been renamed to "database" as it contained the database name.Please use "database" to set the database name.

/tmp/ipykernel_28591/2123065259.py:12 DeprecationWarning: The "schema" arg has been renamed to "database" as it contained the database name.Please use "database" to set the database name.

## From exchangerates api

In [2]:
import os
import importlib

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

# Import the module using importlib
# exchange_rates_api = importlib.import_module('utils.exchange_rates_api')

# Reload the module
# exchange_rates_api = importlib.reload(exchange_rates_api)

from utils.exchange_rates_api import ExchangeRatesApi

In [3]:
exchange_rates_api = ExchangeRatesApi()
# Convert the dictionary to a list of dictionaries with 'code' and 'Description' keys

## Currencies

In [4]:
def get_symbols():
    symbols = pd.DataFrame([
        {'code': key, 'description': value} for key, value in exchange_rates_api.get_symbols()['symbols'].items()
    ])
    already_existing = warehouse_hook.get_pandas_df(
        sql="""
            select name from currency_conversion_rates.currency_dimension where name in %(cn)s
        """,
        parameters={'cn': tuple(symbols['code'].tolist())}
    )['code'].tolist()

    symbols = symbols[~symbols['code'].isin(already_existing)]

    warehouse_hook.insert_rows(
        table='currency_conversion_rates.currency_dimension',
        target_fields=['code', 'description'],
        replace=False,
        rows=tuple(symbols[['code', 'description']].replace({np.nan: None}).itertuples(index=False, name=None)),
        commit_every=100
    )

In [29]:
all_rates_ = []
def get_exchange_rates(base_codes: list):
    exchange_rates_api = ExchangeRatesApi()
    warehoused_codes = warehouse_hook.get_pandas_df(
        sql="""select code from currency_exchange_rates.currency_dimension"""
    )['code'].tolist()
    all_base_code_rates = []

    for base_code in base_codes:
        all_base_code_rates.append(exchange_rates_api.get_latest(
            currency_codes=warehoused_codes,
            base_code=base_code
        ))

    all_rates = []
    for base_code in all_base_code_rates:
        base = base_code['base']
        date = base_code['date']
        timestamp = base_code['timestamp']
        rates = base_code['rates']
        all_rates.extend(
            [{'code': key, 'rate': value, 'base': base, 'date': date, 'timestamp': timestamp} for key, value in
             rates.items()])

    latest = pd.DataFrame(all_rates, columns=['base', 'date', 'timestamp', 'code', 'rate'])
    currency_keys = warehouse_hook.get_pandas_df(
        sql="""
            select code as code_, id as currency_key from currency_exchange_rates.currency_dimension
            where code in %(currency_codes)s
        """,
        parameters={'currency_codes': tuple(latest['code'].unique().tolist())}
    )
    latest = latest.merge(
        currency_keys,
        left_on='base',
        right_on='code_',
        how='inner'
    ).rename(columns={'currency_key': 'base2'}).drop(columns=['code_']).merge(
        currency_keys,
        left_on='code',
        right_on='code_',
        how='inner'
    ).rename(columns={'currency_key': 'code2'}).drop(columns=['code_'])

    return latest.dropna()

    # warehouse_hook.insert_rows(
    #     table='currency_exchange_rates.rates_fact_table',
    #     target_fields=[
    #         'currency_key_1', 'currency_key_2', 'conversion_rate', 'retrieved_on_timestamp', 'retrieved_on_date'
    #     ],
    #     replace=False,
    #     rows=tuple(latest[[
    #         'base', 'code', 'rate', 'timestamp', 'date'
    #     ]].dropna().replace({np.nan: None}).itertuples(index=False, name=None)),
    #     commit_every=0
    # )

conversion_rates = get_exchange_rates(base_codes=['KES', 'USD'])

[2023-04-23 12:01:37,906] {base.py:73} INFO - Using connection ID 'airflow_warehouse_user' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

[2023-04-23 12:01:44,380] {base.py:73} INFO - Using connection ID 'airflow_warehouse_user' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

In [23]:
currency_keys = warehouse_hook.get_pandas_df(
    sql="""
        select code as code_, id as currency_key from currency_exchange_rates.currency_dimension
        where code in %(currency_codes)s
    """,
    parameters={'currency_codes': tuple(conversion_rates['code'].unique().tolist())}
)
currency_keys[currency_keys['code_'] == 'NGN']

[2023-04-23 11:55:08,673] {base.py:73} INFO - Using connection ID 'airflow_warehouse_user' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,code_,currency_key
1,NGN,2


In [30]:
# conversion_rates['base2'] = conversion_rates.merge(currency_keys, left_on='base', right_on='code_', how='inner')['currency_key']
# conversion_rates['code3'] = conversion_rates.merge(currency_keys, left_on='code', right_on='code_', how='inner')['currency_key'].tolist()
#
# # conversion_rates.merge(currency_keys, left_on='code', right_on='code_', how='inner')
#
# # conversion_rates[conversion_rates['code'] == 1]
# conversion_rates[conversion_rates['code2'] == 1]
conversion_rates[conversion_rates['code'] == 'NGN']
# conversion_rates.merge(
#     currency_keys,
#     left_on='code',
#     right_on='code_',
#     how='inner'
# ).rename(columns={'currency_key': 'base3'})

,base,date,timestamp,code,rate,base2,code2
0,KES,2023-04-23,1682240463,RWF,8.297844,3,1
1,USD,2023-04-23,1682240463,RWF,1113.000000,5,1
2,KES,2023-04-23,1682240463,NGN,3.433432,3,2
3,USD,2023-04-23,1682240463,NGN,460.530377,5,2
4,KES,2023-04-23,1682240463,KES,1.000000,3,3
...,...,...,...,...,...,...,...
335,USD,2023-04-23,1682240463,ZMK,9001.203589,5,169
336,KES,2023-04-23,1682240463,ZMW,0.130214,3,170
337,USD,2023-04-23,1682240463,ZMW,17.465772,5,170
338,KES,2023-04-23,1682240463,ZWL,2.400631,3,171
